In [2]:
# import everything that I will use
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from tika import parser

/Users/luizfernandotoledo/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
#open Selenium Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/luizfernandotoledo/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_46381/1903745058.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# this website is a reposity of all federal government decisions related to FOIA requests appeals
# we want to understand how LGPD, the brazilian privacy law, is being used to deny access to public information
# so we make driver open this link with all decisions that mention LGPD at any point
driver.get("http://buscaprecedentes.cgu.gov.br/busca/SitePages/resultadopesquisa.aspx?k=ALL(lgpd)")

In [5]:
#step one: create a list with links from all the pages

#create a list with all links containing CGU decisions related to LGPD, page to page
list_of_links = []
#try to open up to 100 pages in order to find those links
for i in range(100):
    for pedido in driver.find_elements(By.CLASS_NAME, 'ms-srch-ellipsis'):
        links = pedido.find_elements(By.TAG_NAME, 'a')
        for link in links:
            caso = link.get_attribute('href')
            list_of_links.append(caso)
    #click on the next page
    try:
        driver.find_element(By.XPATH, '//*[@id="PageLinkNext"]').click()
        time.sleep(2)
    except:
        break


In [6]:
#test list
print(len(list_of_links))
for link in list_of_links:
    print(link)

222
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/18883000129202128_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/08198029455202191_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137009925202111_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137010897202177_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137022808202054_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137003633202167_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137008313202101_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137004027202169_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137006543202128_CGU.pdf#search=ALL%28lgpd%29
http://buscaprecedentes.cgu.gov.br/busca/dados/Prec

In [29]:
# step2: loop through those links, open the pdfs, read them, get the part of the text where the final decision...
# ...is written and then save it into a list.


# count the decision number
list_of_contents = []
request_n = 0
total_links = len(list_of_links)

# loop through every link, get the text from PDF using Tika, use regex to extract only the parts that I need
# and then only print those parts

for pdf in list_of_links:
    
    try:
        small_list = []
        #calculate how much % of the work has been finished and then print it
        request_n = request_n + 1
        percentage = round(int(request_n) / int(total_links) * 100,2)
        print(f'looking at pdf number {request_n} out of {total_links} or {percentage}%')
        #read the pdf with Tika and extract its content as a text
        url = pdf
        pdfFile = parser.from_file(url)
        final = (pdfFile["content"])
        #remove the spaces from the text extracted
        final = final.replace('\n', ' ')
        #regex to get only specific parts of the document  related to request ID, request, public body name, public official opinion and final decision
        numero_decisao_regex = r"Número do processo:\s(.*)\sÓrgão:"
        numero_decisao = re.findall(numero_decisao_regex, final)[0]
        nome_orgao_regex = r"Órgão:\s(.*)\sAssunto:"
        nome_orgao = re.findall(nome_orgao_regex, final)[0]
        opiniao_tecnica_regex = r"Opinião técnica:\s(.*)\sRELATÓRIO"
        opiniao_tecnica = re.findall(opiniao_tecnica_regex, final)[0]
        pedido_resposta_regex = r"Resumo das manifestações do cidadão:\s\s\sInicial:(.*)\s\s1ª instância:"
        pedido_resposta = re.findall(pedido_resposta_regex, final)[0]
        decisao_regex = r"D E C I S Ã O(.*)Entenda"
        decisao = re.findall(decisao_regex, final)[0]
        # put everything inside a list and then again in another list in order to create a dataframe
        small_list.append(numero_decisao)
        small_list.append(nome_orgao)
        small_list.append(opiniao_tecnica)
        small_list.append(pedido_resposta)
        small_list.append(decisao)
        list_of_contents.append(small_list)
        
    except:
        print(f'For unknown reasons, Regex did not work for this PDF. Maybe it is because of different text patterns')
        #when regex is not found, send all the pdf text to first column in the spreadsheet
        small_list.append(final)
        list_of_contents.append(small_list)
       
        
     

2021-12-07 01:33:37,914 [MainThread  ] [INFO ]  Retrieving http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/18883000129202128_CGU.pdf#search=ALL%28lgpd%29 to /var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/busca-dados-precedente-18883000129202128_cgu.pdf.


looking at pdf number 1 out of 222 or 0.45%


2021-12-07 01:33:39,230 [MainThread  ] [INFO ]  Retrieving http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/08198029455202191_CGU.pdf#search=ALL%28lgpd%29 to /var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/busca-dados-precedente-08198029455202191_cgu.pdf.


looking at pdf number 2 out of 222 or 0.9%


2021-12-07 01:33:40,469 [MainThread  ] [INFO ]  Retrieving http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/00137009925202111_CGU.pdf#search=ALL%28lgpd%29 to /var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/busca-dados-precedente-00137009925202111_cgu.pdf.


...

looking at pdf number 220 out of 222 or 99.1%


2021-12-07 01:39:03,034 [MainThread  ] [INFO ]  Retrieving http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/60143002674202189_CGU.pdf#search=ALL%28lgpd%29 to /var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/busca-dados-precedente-60143002674202189_cgu.pdf.


looking at pdf number 221 out of 222 or 99.55%


2021-12-07 01:39:04,937 [MainThread  ] [INFO ]  Retrieving http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/60143002675202123_CGU.pdf#search=ALL%28lgpd%29 to /var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/busca-dados-precedente-60143002675202123_cgu.pdf.


looking at pdf number 222 out of 222 or 100.0%


In [30]:
#check if we've got everything
count_link = len(list_of_links)
count_content = len(list_of_contents)

print(f'There are {count_link} links and {count_content} contents.')

There are 222 links and 222 contents.


In [31]:
#test what is inside each content
list_of_contents

[['18883.000129/2021-28 ',
  'Banco do Nordeste do Brasil S.A. - BNB  ',
  ' Opina-se pelo conhecimento e, no mérito, pelo desprovimento do recurso devido ao sigilo bancário, nos termos do art. 22 da Lei n° 12.527/2011, c/c o art. 1º da Lei Complementar n° 105/2001 e, ainda, à preservação de competitividade do recorrido, nos termos do art. 5°, § 2° do Decreto n° 7.724/2012, c/c o art. 22 da Lei n° 12.527/2011.    ',
  ' Solicitou informações sobre empréstimos concedidos com recursos do FNE.  1ª instância: Reiterou o pedido reduzindo seu escopo. Sendo assim, a lista solicitada deverá conter os seguintes dados: nome do beneficiário, CPF e o município de localização do empreendimento financiado dos anos de 2018, 2019, 2020 a 09/2021 (lista separada por município). Alegou que o sigilo bancário não deve ser aplicado neste caso, devido aos recursos serem públicos.  2ª instância: Reiterou o pedido. Alegou que o banco já fornece essas informações de forma parcial em seu site na internet. Escla

In [32]:
import pandas as pd

df = pd.DataFrame(list_of_contents, columns = ['num_pedido', 'orgao', 'opiniao_tecnica', 'pedido_e_resposta_do_orgao', 'decisao_cgu'])
df

,num_pedido,orgao,opiniao_tecnica,pedido_e_resposta_do_orgao,decisao_cgu
0,18883.000129/2021-28,Banco do Nordeste do Brasil S.A. - BNB,"Opina-se pelo conhecimento e, no mérito, pelo...",Solicitou informações sobre empréstimos conce...,No exercício das atribuições a mim conferida...
1,08198.029455/2021-91,Ministério da Justiça e Segurança Pública - MJSP,Opina-se pelo não conhecimento do recurso devi...,"Solicita lista de informações, em formato abe...",No exercício das atribuições a mim conferida...
2,00137.009925/2021-11,Gabinete de Segurança Institucional da Presidê...,"Opina-se: a) pelo conhecimento e, no mérito...",o requerente solicita os registros de entrada...,No exercício das atribuições a mim conferida...
3,00137.010897/2021-77,Gabinete de Segurança Institucional da Presidê...,"Opina-se: a.1) pelo não conhecimento, nos te...",o requerente solicita (a) lista com nomes e o...,No exercício das atribuições a mim conferida...
4,00137.022808/2020-54,Gabinete de Segurança Institucional da Presidê...,"Opina-se pelo conhecimento do recurso e, no ...",O requerente deseja obter informações sobre o...,No exercício das atribuições a mim conferi...
...,...,...,...,...,...
217,60143.002749/2021-21,Comando do Exército - CEX,"Opina-se pelo conhecimento do recurso e, no ...",O cidadão solicita a íntegra do processo admi...,No exercício das atribuições a mim conferida...
218,60143.002769/2021-01,Comando do Exército - CEX,"Opina-se pelo conhecimento do recurso e, no ...",O cidadão solicita acesso à íntegra dos docum...,No exercício das atribuições a mim conferida...
219,60143.002645/2021-17,Comando do Exército - CEX,"Opina-se pelo conhecimento do recurso e, no ...",O cidadão solicita acesso à íntegra do proces...,No exercício das atribuições a mim conferida...
220,60143.002674/2021-89,Comando do Exército - CEX,Opina-se: pelo conhecimento e provimento pa...,O cidadão solicita acesso às seguintes inform...,No exercício das atribuições a mim conferida...


In [33]:
#save to a csv file
df.to_csv('decisoes_da_cgu_que_citam_lgpd.csv')

In [12]:
### space to create more regex if you want and then include into the original code
# text_sample is the full txt of one PDF decision so you can create everything from there


In [13]:
# text_sample = '''

# processo-08198029455202191   CONTROLADORIA-GERAL DA UNIÃO  LEI DE ACESSO À INFORMAÇÃO - RECURSO SUBMETIDO À CGU    PARECER N° 1459/2021/CGRAI/OGU/CGU  Número do processo: 08198.029455/2021-91  Órgão: Ministério da Justiça e Segurança Pública - MJSP  Assunto: Recurso contra negativa a pedido de acesso à informação.   Data do Recurso à CGU: 19/10/2021  Restrição de acesso no recurso à CGU (Fala.BR): Não.  Requerente: Não identificado.  Opinião técnica: Opina-se pelo não conhecimento do recurso devido à inexistência da informação, nos termosda Súmula CMRI nº 6/2015.     RELATÓRIO   Resumo das manifestações do cidadão:   Inicial: Solicita lista de informações, em formato aberto, anteriormente divulgadas no site do órgão, que tenham sido removidas em virtude/com fundamento na edição e vigência da Lei Geral de Proteção de Dados.  1ª instância:  Esclarece que seu pedido trata-se de saber se o órgão removeu alguma informação do seu site com base na LGPD e, caso sim, quais, de modo que seja fornecida a lista dessas informações.  2ª instância:  Refuta as argumentações e esclarece que seu interesse não é o dado em si.  Respostas do órgão:   Inicial: Classifica o pedido como sendo genérico e nega acesso.  1ª instância: Reitera a negativa do pedido por entender que se trata de pedido genérico apontando a de especificação da informação requerida, nos termos do Art.10, da Lei nº 12.527, de 2011.  2ª instância: Ratifica a negativa nos termos apresentados anteriormente.  Resumo do Recurso à CGU:    Reitera o pedido nos termos apresentados anteriormente e acrescenta que obteve resposta em pedido idêntico direcionado ao Ministério da Defesa.  Instrução do Recurso:   A instrução processual levou em consideração as informações constantes da Plataforma Fala.BR, os esclarecimentos adicionais recebidos pelo recorrido, além de observar as determinações da LAI e de sua regulamentação, bem como da legislação específica aplicável à matéria e precedentes desta Casa.  Análise    1. O presente recurso trata de pedido de acesso à informação direcionado ao Ministério da Justiça e Segurança Pública - MJSP, em que o requerente solicita lista, em formato aberto, de informações anteriormente divulgadas no site do órgão, que tenham sido removidas em virtude/com fundamento na edição e vigência da Lei Geral de Proteção de Dados.  2. A resposta inicial do MJSP, reiterada nas instâncias recursais, classifica o pedido do requerente como sendo genérico nos termos do  art. 13, do Decreto n. 7.724/2012, sob o argumento de que o pedido de acesso à informação deve conter especificação, de forma clara e precisa, da informação solicitada.  Parecer - Recurso de 3ª Instância 1459 (2197620)         SEI 08198.029455/2021-91 / pg. 1    3. O requerente por sua vez, nas instâncias recursais esclarece que seu pedido é claro, de modo seu interesse é saber se o órgão removeu alguma informação do seu site com base na LGPD e, caso sim, quais, por meio de uma lista. No recurso à CGU, acrescenta que realizou pedido idêntico (60110.003477/2021-64) para o Ministério da Defesa e conforme resposta do órgão, não foi necessário a remoção ou até mesmo a adequação que qualquer conteúdo em decorrência da Lei, visto que as informações disponíveis no sítio do MD respeitavam os limites impostos pela LAI.   4. Da leitura do caso em tela, apenas para contextualização, verifica-se que além do pedido direcionado ao MD indicado pelo requerente, também tramitou nesta casa, pedido do requerente com manifestação idêntica direcionado ao Ministério da Ciência, Tecnologia, Inovações - MCTI (01217.006788/2021-28), o qual, após tratativas com o órgão, não foi conhecido pela CGU, visto não existir informações removidas no site do órgão, em decorrência da LGPD.  5. Passando a análise do caso concreto, compreende-se que embora o pedido inicial do requerente se apresente de forma ampla, em primeira instância, os argumentos apresentados por ele norteiam o objeto de seu interesse, o qual limita-se saber quais informações foram removidas em virtude da vidência da LGPD. Situação que não envolve acesso ao dado em si, mas apenas saber e obter lista de possíveis informações que foram retiradas do site em decorrência da LGPD, afastando a compreensão de que se trata de pedido genérico.   6. Em sede de esclarecimentos adicionais, o MJSP apresentou posicionamento de suas áreas sobre a solicitação inicial do requerente. Todas as áreas consultadas apontaram não ter sido necessário  remover ou solicitar a remoção de quaisquer dados/informações com a edição da Lei nº 13.853, de 8 de julho de 2019, conforme destaques:    DESPACHO Nº 1819/2021/DTIC/SE "...Diretoria de Tecnologia da Informação e Comunicação da Secretaria- Executiva - DTIC/SE não realiza a retirada ou inserção de informação do site oficial deste Ministério. As atribuições desta Diretoria se referem à manutenção da infraestrutura para funcionamento do referido serviço web."   DESPACHO Nº 1706/2021/DEP/SEGEN... Diretor de Ensino e Pesquisa   DESPACHO Nº 7103/2021/DIGES/SEGEN "...Nesse sentido, informo que esta Diretoria de Gestão não possui, nem é responsável por publicações que divulgam dados pessoais de pessoa natural ou pessoa jurídica de direito público ou privado. Desse modo, não houve a necessidade de remover ou solicitar a remoção de quaisquer dados/informações com a edição da Lei nº 13.853, de 8 de julho de 2019, que alterou a Lei nº 13.709, de 14 de agosto de 2018."   DESPACHO Nº 532/2021/CGPGC/GAB-SENAJUS/SENAJUS "...Informamos que todos os instrumentos celebrados nesta Coordenação- Geral de Planejamento e Gestão de Convênios são acompanhados e publicados por meio da Plataforma +BRASIL, de acesso público, bem como publicados no Diário Oficial da União os respectivos extratos."   DESPACHO Nº 3926/2021/GAB-DEMIG/DEMIG/SENAJUS "...informo que nenhuma informação foi removida das páginas eletrônicas geridas por este Departamento de Migrações."   DESPACHO Nº 89/2021/SIC-SENAD/SENAD INFORMAÇÃO Nº 8/2021/CGINVEST/DPPA/SENAD INFORMAÇÃO Nº 2/2021/CTPAS/CGAT/DGA/SENAD "...informamos que esta unidade não solicitou remoção de nenhuma informação de suas páginas do site institucional do MJSP e LGPD. esta Diretoria de Gestão de Ativos não identificou remoção de informações do sítio do MJSP, que sejam relacionadas à gestão de ativos, em virtude da/com fundamento na edição e vigência da Lei Geral de Proteção de Dados. Ressalta-se que as informações gerenciais disponíveis no sítio do MJSP e relacionadas à gestão de ativos são públicas e impessoais. Cabe destacar, ainda, que buscamos sempre aprimorar os dados afetos aos processos desenvolvidos no âmbito desta Diretoria, a exemplo da recente inclusão do Portifólio da Gestão de Ativos, em transparência ativa, na página eletrônica da SENAD."   DESPACHO Nº 2240/2021/GAB-DPDC/DPDC/SENACON DESPACHO Nº 952/2021/CGEMM/DPDC/SENACON (16500388), DESPACHO Nº 718/2021/CGSINDEC/DPDC/SENACON (16519089) e INFORMAÇÃO Nº 24/2021/CGCTSA/DPDC/SENACON (16529739). "...informo que este Departamento não realizou a retirada de informações do site, em virtude da LGPD..."  Parecer - Recurso de 3ª Instância 1459 (2197620)         SEI 08198.029455/2021-91 / pg. 2  http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/Item/displayifs.aspx?List=0c839f31%2D47d7%2D4485%2Dab65%2Dab0cee9cf8fe&ID=1575914&Source=http%3A%2F%2Fwww%2Econsultaesic%2Ecgu%2Egov%2Ebr%2Fbusca%2FSitePages%2Fresultadopesquisa%2Easpx%3Fk%3D60110%252E003477%252F2021%252D64%252C&Web=88cc5f44%2D8cfe%2D4964%2D8ff4%2D376b5ebb3bef http://buscaprecedentes.cgu.gov.br/busca/dados/Precedente/01217006788202128_CGU.pdf#search=01217%2E006788%2F2021%2D28     DESPACHO Nº 567/2021/SIC-SENASP/SENASP "...Informo que, no âmbito desta Senasp, não foram retiradas informações do sítio eletrônico do órgão em virtude da Lei Geral de Proteção de Dados - LGPD."   DESPACHO Nº 2230/2021/DPJUS/SENAJUS "Considerando o contido nos despachos (16540958, 16521097 e 16493798), informo que no âmbito deste Departamento não foram retiradas informações do sítio eletrônico do órgão em virtude da Lei Geral de Proteção de Dados - LGPD."   DESPACHO Nº 203/2021/SIC-SEGEN/GAB-SEGEN/SEGEN "...no âmbito da Secretaria de Gestão e Ensino em Segurança Pública não foram retiradas informações do sítio eletrônico do órgão em virtude da Lei Geral de Proteção de Dados - LGPD,  para envio de resposta ao cidadão."   INFORMAÇÃO Nº 125/2021/SIC-GM/GM \'Em atendimento ao requerido, esclarece-se que a Assessoria de Comunicação Social do Gabinete do Ministro, por meio do Despacho nº  270/2021/ASCOM/GM (16483032), informou que:  (...) “Em manifestação ao pedido, destacamos que em relação aos conteúdos(notícias, áudios releases, vídeos) de responsabilidade da Ascom - Assessoria de Comunicação Social,  não temos conhecimento de nenhum pedido para remoção de conteúdo com base na LGPD (Lei Geral de Proteção de Dados Pessoais).” (...) Assim sendo, tendo em vista a informação de que, por parte desta unidade não houveram, em virtude da Lei Geral de Proteção de dados - LGPD, documentos excluídos do site do Ministério da justiça e Segurança Pública, restitua-se o processo ao Serviço de Informação ao Cidadão – SIC Central, para os impulsos decorrentes.\'    INFORMAÇÃO Nº 1/2021/SIC-SEOPI/SEOPI ."...que esta Secretaria de Operações Integradas somente realiza a "inserção" dos dados da AGENDA dos dirigentes, não tendo retirado quaisquer dados do site oficial deste Ministério, em virtude da LGPD.   INFORMAÇÃO Nº 3/2021/ASSE-DRCI/DRCI/SENAJUS "A esse respeito, este Departamento informa que não realizou a retirada de informações do portal do Ministério da Justiça e Segurança Pública em virtude da LGPD, uma vez que as informações publicadas por esta unidade possuem caráter informativo e são feitas em estrito cumprimento legal."   OFÍCIO Nº 1817/2021/SAA/SE/MJ "A esse respeito, subsidiado pela Coordenação-Geral de Arquitetura e Engenharia - CGAE, Despacho 386 (SEI nº 16493507); Coordenação-Geral de Gestão Documental e Serviços Gerais - CGDS, Ofício 680 (SEI nº 16506189); Coordenação-Geral de Gestão de Pessoas - CGGP, Informação 13 (SEI nº 16502787); Coordenação-Geral de Licitações e Contratos - CGL, Despacho 2624 (SEI nº 16503563), informo que, no âmbito de atuação da Subsecretaria de Administração, as informações que compõem a transparência ativa são referentes a:  Lista quadrimestral de terceirizados deste Ministério: publicação da lista de terceirizado de acordo com determinação do art. 133 da Lei nº 13.408/2016, em conformidade com a Política de Dados Abertos do Poder Executivo federal, instituída pelo Decreto nº 8.777, de 2016. Além disso, essa lista é também encaminhada à Controladoria-Geral da União. Link de acesso: https://dados.mj.gov.br/dataset/lista-de-terceirizados- do-ministerio-da-justica; Licitações e contratos: as contratações realizadas pelo Ministério da Justiça e Segurança Pública são publicadas no sítio eletrônico, atendendo à transparência ativa estabelecida na Lei nº 12.527/2011, em especial ao Art. 8º, e podem ser acompanhadas pelo cidadão: https://www.gov.br/mj/pt-br/acesso-a-informacao/licitacoes- e-contratosv1; Pessoal: os dados de pessoal estão disponíveis de forma centralizada nos Portais do Governo Federal, que disponibiliza informações e dados por tema (perfil de servidores, cargos e carreiras, painéis com dados diversos, Estatísticas de Pessoal, etc). Assim, a gestão específica de publicação desses dados não fica a cargo da unidade de gestão de pessoas desta unidade.  Em resposta aos questionamentos do requerente, cumpre esclarecer que os dados divulgados por esta Subsecretaria de Administração decorrem de obrigação legal, cumpridas em função das legislações pertinentes a cada temática. Assim, não  foram retiradas quaisquer informações das páginas do sítio desta Pasta, no que tange ao escopo de matérias de competências desta unidade,  portanto inexiste informações a serem prestadas nos moldes solicitados."   INFORMAÇÃO Nº 6/2021/GAB-SENACON/SENACON "...informamos que o Gabinete da Secretaria Nacional do Consumidor não realizou a retirada de informações do site em virtude da LGPD,..."  Parecer - Recurso de 3ª Instância 1459 (2197620)         SEI 08198.029455/2021-91 / pg. 3  https://dados.mj.gov.br/dataset/lista-de-terceirizados-do-ministerio-da-justica http://www.planalto.gov.br/ccivil_03/_Ato2011-2014/2011/Lei/L12527.htm https://www.gov.br/mj/pt-br/acesso-a-informacao/licitacoes-e-contratosv1     INFORMAÇÃO Nº 51/2021/SIC-SENACON/SENACON "A esse respeito, subsidiada pela Secretaria Nacional do Consumidor, encaminho em anexo os documentos Sei nº (s) 16529925 e 16556292, produzidos pelos Gabinetes da Secretaria Nacional do Consumidor, referente ao caso em tela."    7. Diante do exposto pelo Órgão, é possível compreender que a informação pleiteada pelo requerente, atualmente, é inexistente no âmbito do órgão, visto que  não foram removidas de seu  site informações em decorrência da LGPD. Nesse sentido, entende-se pela aplicação do disposto na  Súmula CMRI nº 6/2015- Inexistência de Informação.  Conclusão   8. De todo o exposto, portanto, opina-se pelo não conhecimento do recurso devido à inexistência da informação, nos termos da Súmula CMRI nº 6/2015.  9. À consideração superior.     GABRIELA NOGUEIRA CUNHA FAMBRE GONÇALVES   Analista Administrativo           CGU  Controladoria-Geral da União  Ouvidoria-Geral da União  Coordenação-Geral de Recursos de Acesso à Informação     D E C I S Ã O  No exercício das atribuições a mim conferidas pelo Decreto nº 9.681, de 03 de janeiro de 2019, adoto, como fundamento deste ato, nos termos do art. 23 do Decreto nº 7.724/2012, o parecer anexo, para decidir pelo�não conhecimento�do recurso interposto, no âmbito do pedido de informação 08198.029455/2021-91, direcionado ao Ministério da Justiça e Segurança Pública - MJSP.      FÁBIO DO VALLE VALGAS DA SILVA  Ouvidor-Geral da União – Adjunto     Entenda a decisão da CGU:   Não conhecimento - O recurso não foi analisado no mérito pela CGU, pois não atende a algum requisito que permita essa análise: a informação foi declarada inexistente pelo órgão, o pedido não pode ser atendido por meio da Lei de Acesso à Informação, a informação está classificada, entre outros.   Perda (parcial) do objeto - A informação solicitada (ou parte dela) foi disponibilizada pelo órgão antes da decisão da CGU, usualmente por e-mail. A perda do objeto do recurso também é reconhecida nos casos em que�o órgão se compromete a disponibilizar a informação solicitada (ou parte dela)�ao requerente em ocasião futura, indicando prazo, local e modo de acesso.   Desprovimento - O acesso à informação solicitada não é possível, uma vez que as razões apresentadas pelo órgão para negativa de acesso possuem fundamento legal.   Provimento (parcial) – A CGU determinou a entrega da informação (ou de parte dela) ao cidadão.      Conheça mais sobre a Lei de Acesso à Informação:   Portal “Acesso à Informação”  Parecer - Recurso de 3ª Instância 1459 (2197620)         SEI 08198.029455/2021-91 / pg. 4    https://www.gov.br/acessoainformacao/pt-br       Publicação “Aplicação da Lei de Acesso à Informação na Administração Pública Federal”   https://www.gov.br/acessoainformacao/pt-br/central-de-conteudo/publicacoes/arquivos/aplicacao- da-lai-2019.pdf      Decisões da CGU e da CMRI   http://buscaprecedentes.cgu.gov.br/busca/SitePages/principal.aspx       Busca de Pedidos e Respostas da LAI:   https://www.gov.br/acessoainformacao/pt-br/assuntos/busca-de-pedidos-e-respostas/busca-de- pedidos-e-respostas       Documento assinado eletronicamente por GABRIELA NOGUEIRA CUNHA FAMBRE GONÇALVES, Analista Administrativo, em 01/12/2021, às 09:25, conforme horário oficial de Brasília, com fundamento no § 3º do art. 4º do Decreto nº 10.543, de 13 de novembro de 2020.  Documento assinado eletronicamente por FABIO DO VALLE VALGAS DA SILVA, Ouvidor-Geral da União, Adjunto, em 02/12/2021, às 15:05, conforme horário oficial de Brasília, com fundamento no § 3º do art. 4º do Decreto nº 10.543, de 13 de novembro de 2020.   A autenticidade deste documento pode ser conferida no site  https://sei.cgu.gov.br/conferir informando o código verificador 2197620 e o código CRC 8D5CE21F Referência: Processo nº 08198.029455/2021-91 SEI nº 2197620  Parecer - Recurso de 3ª Instância 1459 (2197620)         SEI 08198.029455/2021-91 / pg. 5  https://www.gov.br/acessoainformacao/pt-br https://www.gov.br/acessoainformacao/pt-br/central-de-conteudo/publicacoes/arquivos/aplicacao-da-lai-2019.pdf http://buscaprecedentes.cgu.gov.br/busca/SitePages/principal.aspx https://www.gov.br/acessoainformacao/pt-br/assuntos/busca-de-pedidos-e-respostas/busca-de-pedidos-e-respostas  \tParecer - Recurso de 3ª Instância 1459 (2197620)\n\n'

# '''




In [14]:
### test regex

# new_regex = r"write_your_regex"
# new = re.findall(new_regex, text_sample)[0]